In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# display("text/html", "<style>.container { width:100% !important; }</style>")
display(HTML(""))

In [9]:
import pandas as pd 
import collections
import numpy as np
import math

# un premier travail a ete fait sur le fichier csv de depart.
# J'ai supprime toutes les lignes pour lesquelles upper depth et lower depth sont simultanement 
# superieures a 30 cm a causes des depths standards de ISRIC contrainte nommee let30

# Maintenant cette 1ere contrainte (let30) a ete levee dans la mise a jour de notre programme

# we also suggest that the original file contains no missing values for all the parameters that are being studied

# Ensuite il faut confirmer que pour chaque profil le lower depth de horizon precedent (ligne i+1) est egal au upper depth de horizon suivant  (ligne i))

#this function retrieves the value of a parameter in the original dataframe
def select_val(profil_arg, profondeur_index, param_index):
        return profil_arg.iloc[profondeur_index, param_index]

#this function returns the H+ concentration corresponding to a pH value given in argument
def concentr(pH):
    return pow(10,-pH)
    
#this function returns the sign of the difference between 2 quantities a and b, in this order!
# but we have not used it anymore
def sign_delta(a,b):
    if (a<b):
        return 1
    else:
        return (-1)

#the following 3 functions are used in the case selector function right following
def A(x):
    return (x<=5)
def B(x):
    return (5<x<=15)
def C(x):
    return (15<x<=30)
# la function D(x) a ete prevue en cas de qu'est ce qu'il y a. Ici, les profondeurs a comparer ne peuvent pas etre sup a 30
def D(x):
    return (x>30)

#this function selects a case checking the conditions on x3, x2 and x4
# les cas 3, 6 et 7 sont impossibles a cause des ordres de comparaison de x3 et x2
def case_selector(x3,x2):
    if (A(x3) and A(x2)):
        return 0
    elif (A(x3) and B(x2)):
        return 1
    elif (A(x3) and C(x2)):
        return 2
    elif (B(x3) and A(x2)):
        print('ce cas est impossible B(x3) and A(x2)')
        return 3
    elif (B(x3) and B(x2)):
        return 4
    elif (B(x3) and C(x2)):
        return 5
    elif (C(x3) and A(x2)):
        print('ce cas est impossible C(x3) and A(x2)')
        return 6
    elif (C(x3) and B(x2)):
        print('ce cas est impossible C(x3) and B(x2)')
        print(x3,x2)
        return 7
    elif (C(x3) and C(x2)):
        return 8 

# Driver function
if __name__ == "__main__":
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 500)
    pd.set_option('display.max_colwidth', None)
#     pd.options.display.width = None
    
    # pre-processing
    df = pd.read_csv('merge_horizons_test.csv')
      
    df_final = df.iloc[0:0]
    
    df['NoH']='' #creation and initialization of the Number of Horizons column
    
    df['Comments']='' #creation and initialization of the comments column for profiles with x1 less than 30 cm
    
#     myFileName=r'C:\Users\David DEGBEY\Documents\DdeclaRe_test\test.xlsx'
    #load the workbook, and put the sheet into a variable
#     wb = load_workbook(filename=myFileName)
#     ws = wb['Sheet1']
    
    #getting all the profile names in a list
    ProfileName_df = df.iloc[:,1]
    my_list = ProfileName_df.values.tolist()
    
    #counting the number of horizons per profile using a collection object (subclass of dictionnary)
    c = collections.Counter(my_list)
    
#     # print the results
#     print("The respective no. of horizons per profile is: ")
#     for key, value in c.items():
#         result = "Profile: {P_name}, No. of horizons: {Num_Hor}".format(P_name = key , Num_Hor = value)
#         print (result)
    
    # adding the no of horizons on each line of the dataframe
    for key, value in c.items():
        df['NoH'] = np.where(df.profil_id == key, value, df.NoH)
    
         
   #print(df.head(100)) 

    list_of_profil = []
    recupx3 = []
    recupx2 = []
    recupx1 = []
    
    # creating a list of profiles for each profil_id
        # we keep only the profiles for which the last "lower depth" is greater or equal to 30 cm
        # we then loose a lot of information (pprofile with one forizon less than 30 cm are all lost)
        
        # later we can think of designing a system which is regardless of the maximum lower depth of observation
        # and that retrieves the maximum information then.
    #creation of the list of profiles excluding profiles with last depths less than 30 cm
#     grouped = df.groupby('profil_id')
#     for profil_id, group in grouped:
#         if ((group.iloc[0,13])>=30.0):
#             list_of_profil.append(group)

    #creation of the list of profiles excluding no profiles
    grouped = df.groupby('profil_id')
    for profil_id, group in grouped:
        list_of_profil.append(group)
    

    # Here begins the creation of the 3 new (harmonized to ISRIC) rows (for 5, 15 and 30 cm) for each profile 
    
    
    for profil in list_of_profil:
        
        recupx3 = []
        recupx2 = []
        recupx1 = []
        tempx3 = [] 
        tempx2 = []
        tempx1 = []
        res = 0 # initialization of temporary variables

        if (profil.iloc[0,15]==1): # if NoH == 1, one horizon

        #here we should add the 3 standardized depths 5, 15, and 30 cm as new rows
#                     print('nous sommes dans ce cas1')   
                    
#                   inititialization of the 3 additional rows
                    new_row5cm = profil.iloc[0,:].copy()
                    new_row15cm = profil.iloc[0,:].copy()
                    new_row30cm = profil.iloc[0,:].copy()
            
                    x1 = profil.iloc[0,13]
                    
#                   updating and replacing each of the 3 additional rows 

                    if (profil.iloc[0,13]>=30): #(if lower depth is greater than 30 cm)
                        new_row5cm['lower_d'] = '5cm'
                        new_row5cm_df = new_row5cm.to_frame().transpose()
                        profil = pd.concat([profil, new_row5cm_df], ignore_index=True)

                        new_row15cm['lower_d'] = '15cm'
                        new_row15cm_df = new_row15cm.to_frame().transpose()
                        profil = pd.concat([profil, new_row15cm_df], ignore_index=True)

                        new_row30cm['lower_d'] = '30cm'
                        new_row30cm_df = new_row30cm.to_frame().transpose()
                        profil = pd.concat([profil, new_row30cm_df], ignore_index=True)
                        
                        df_final = pd.concat([df_final, profil.tail(3)], ignore_index=True)       
                        
                    elif (profil.iloc[0,13]>=15):
                        
                        new_row5cm['lower_d'] = '5cm'
                        new_row5cm_df = new_row5cm.to_frame().transpose()
                        profil = pd.concat([profil, new_row5cm_df], ignore_index=True)

                        new_row15cm['lower_d'] = '15cm'
                        new_row15cm['Comments'] = 'valid from 15 cm to ' + str(x1) + ' cm depth and not beyond !' 
                        new_row15cm_df = new_row15cm.to_frame().transpose()
                        profil = pd.concat([profil, new_row15cm_df], ignore_index=True)
                        
                        df_final = pd.concat([df_final, profil.tail(2)], ignore_index=True)
                        
                    elif (profil.iloc[0,13]>=5):
                        
                        new_row5cm['lower_d'] = '5cm'
                        new_row5cm['Comments'] = 'valid from 5 cm to ' + str(x1) + ' cm depth and not beyond !' 
                        new_row5cm_df = new_row5cm.to_frame().transpose()
                        profil = pd.concat([profil, new_row5cm_df], ignore_index=True)
                        
                        df_final = pd.concat([df_final, profil.tail(1)], ignore_index=True)
                           
                    else: 
                        
                        #print('case of only one depth which is less than 5 cm, no harmonisation possible')
                        
                        pass
                    
                     
        elif (profil.iloc[0,15]==2): #if NoH == 2
            
            #profil.to_csv('exemple.csv', index=False)
            
            x2 = int(profil.iloc[1,13]) #1ere prof en quittant le sol niveau 0
            #print(type(x2))
            x1 = int(profil.iloc[0,13]) #2eme prof en quittant le sol niveau 0
                        
            for i in range(16):
                if (i not in [0,1,12,13,14,15]):
                    recupx2.append(float(select_val(profil, 1, i))) # recuperation des valeurs de parametres correspondants a la 1ere profondeur
                    recupx1.append(float(select_val(profil, 0, i))) # recuperation des valeurs de parametres correspondants a la 2eme profondeur
                
                else:
                    recupx2.append((select_val(profil, 1, i))) 
                    recupx1.append((select_val(profil, 0, i)))
                
            tempx2 = recupx2.copy()
            tempx1 = recupx1.copy()
                                     
#           inititialization of the 3 additional rows
            new_row5cm = profil.iloc[0,:].copy()
            new_row15cm = profil.iloc[0,:].copy()
            new_row30cm = profil.iloc[0,:].copy()
            
            if (x2<=5): #if x2 is less than 5
                for j in range(16):
                    
                    if(j not in [0,1,12,13,14,15]):
                        new_row15cm[j] = recupx1[j]
                        new_row30cm[j] = recupx1[j]
                        if (j not in [4,5]): #if the column j is not a pH column
                            new_row5cm[j] = (x2*recupx2[j]+(5-x2)*recupx1[j])/5
                        else:
                            tempx2[j] = concentr(recupx2[j])
                            tempx1[j] = concentr(recupx1[j])
                            res = (x2*tempx2[j]+(5-x2)*tempx1[j])/5
                            new_row5cm[j] = (-1)*(math.log10(res))
                            
                
            elif (x2<=15): #if x2 is less than 15
                for j in range(16):
                    if(j not in [0,1,12,13,14,15]):
                        new_row5cm[j] = recupx2[j]
                        new_row30cm[j] = recupx1[j]
                        if (j not in [4,5]): #if the column j is not a pH column
                            new_row15cm[j] = ((x2-5)*recupx2[j]+(15-x2)*recupx1[j])/10
                        else:
                            tempx2[j] = concentr(recupx2[j])
                            tempx1[j] = concentr(recupx1[j])
                            res = ((x2-5)*tempx2[j]+(15-x2)*tempx1[j])/10
                            new_row15cm[j] = (-1)*(math.log10(res))
                            
            elif (x2<=30): #if x2 is less than 30
                for j in range(16):
                    if(j not in [0,1,12,13,14,15]):
                        new_row5cm[j] = recupx2[j]
                        new_row15cm[j] = recupx2[j]
                        if (j not in [4,5]): #if the column j is not a pH column
                            new_row30cm[j] = ((x2-15)*recupx2[j]+(30-x2)*recupx1[j])/15
                            
                        else:
                            tempx2[j] = concentr(recupx2[j])
                            tempx1[j] = concentr(recupx1[j])
                            res = ((x2-15)*tempx2[j]+(30-x2)*tempx1[j])/15
                            new_row30cm[j] = (-1)*(math.log10(res))
                            
            else:
                pass
            

            
#             profil = profil.append(new_row5cm, ignore_index=True)
                    

#             profil = profil.append(new_row15cm, ignore_index=True)
                    
            new_row30cm['lower_d'] = '30cm'
            new_row30cm_df = new_row30cm.to_frame().transpose()
            profil = pd.concat([profil, new_row30cm_df], ignore_index=True)
                    
            new_row15cm['lower_d'] = '15cm'
            new_row15cm_df = new_row15cm.to_frame().transpose()
            profil = pd.concat([profil, new_row15cm_df], ignore_index=True)
            
#           updating and replacing each of the 3 additional rows 
            new_row5cm['lower_d'] = '5cm'
            new_row5cm_df = new_row5cm.to_frame().transpose()
            profil = pd.concat([profil, new_row5cm_df], ignore_index=True)
            
            if (x1>=30):
                df_final = pd.concat([df_final, profil.tail(3)], ignore_index=True) #ajout des 3 dernieres lignes comme valeurs harmonisees
            elif (x1>=15):
                profil.iloc[3,16] = 'valid from 15 cm to ' + str(x1) + ' cm depth and not beyond !' 
                df_final = pd.concat([df_final, profil.tail(2)], ignore_index=True) #ajout des 2 dernieres lignes comme valeurs harmonisees
            elif (x1>=5):
                profil.iloc[4,16] = 'valid from 5 cm to ' + str(x1) + ' cm depth and not beyond !'
                df_final = pd.concat([df_final, profil.tail(1)], ignore_index=True) #ajout de la derniere ligne comme valeurs harmonisees       
            else :
                print('impossible because x2 < x1 and previous conditions were made on x2')
                pass
            
                 
                            
        elif (profil.iloc[0,15]==3): # NoH==3
            
            
            x3 = int(profil.iloc[2,13]) #1ere prof en quittant le sol niveau 0
            x2 = int(profil.iloc[1,13]) #2eme prof en quittant le sol niveau 0
            x1 = int(profil.iloc[0,13]) #3eme prof en quittant le sol niveau 0
            
            for i in range(16):
                if (i not in [0,1,12,13,14,15]):
                    try:
                        recupx3.append(float(select_val(profil, 2, i))) # recuperation des valeurs de parametres correspodants a la 1ere profondeur                        
                        recupx2.append(float(select_val(profil, 1, i))) # recuperation des valeurs de parametres correspodants a la 2eme profondeur
                        recupx1.append(float(select_val(profil, 0, i))) # recuperation des valeurs de parametres correspodants a la 3eme profondeur
                    except (ValueError):
                        print ("error on line",i)
                    
                else:
                    recupx3.append((select_val(profil, 2, i)))
                    recupx2.append((select_val(profil, 1, i))) 
                    recupx1.append((select_val(profil, 0, i)))
            
            tempx3 = recupx3.copy()
            tempx2 = recupx2.copy()
            tempx1 = recupx1.copy()
            
#           inititialization of the 3 additional rows
            new_row5cm = profil.iloc[0,:].copy()
            new_row15cm = profil.iloc[0,:].copy()
            new_row30cm = profil.iloc[0,:].copy()

#           in this section we cover all the cases of combination for x3 and x2 (end befor elif NoH ==4)
#           for that we create 3 boolean variables A B C. Kind of truth table

            if (case_selector(x3,x2)==0):
                for j in range(16):
                    if(j not in [0,1,12,13,14,15]):
                        new_row15cm[j] = recupx1[j]
                        new_row30cm[j] = recupx1[j]
                        if (j not in [4,5]): #if the column j is not a pH column
                            new_row5cm[j] = (x3*recupx3[j]+(x2-x3)*recupx2[j]+(5-x2)*recupx1[j])/5
                        else:
                            tempx3[j] = concentr(recupx3[j])
                            tempx2[j] = concentr(recupx2[j])
                            tempx1[j] = concentr(recupx1[j])
                            res = (x3*tempx3+(x2-x3)*tempx2[j]+(5-x2)*tempx1[j])/5
                            new_row5cm[j] = (-1)*(math.log10(res))
                        
                
            elif (case_selector(x3,x2)==1):
                for j in range(16):
                    if(j not in [0,1,12,13,14,15]):
                        new_row30cm[j] = recupx1[j]
                        
                        if (j not in [4,5]): #if the column j is not a pH column
                            new_row5cm[j] = (x3*recupx3[j]+(5-x3)*recupx2[j])/5
                            new_row15cm[j] = ((x2-5)*recupx2[j]+(15-x2)*recupx1[j])/10
                        else:
                            tempx3[j] = concentr(recupx3[j])
                            tempx2[j] = concentr(recupx2[j])
                            tempx1[j] = concentr(recupx1[j])
                            res = (x3*tempx3[j]+(5-x3)*tempx2[j])/5
                            new_row5cm[j] = (-1)*(math.log10(res))
                            res = ((x2-5)*tempx2[j]+(15-x2)*tempx1[j])/10
                            new_row15cm[j] = (-1)*(math.log10(res))
                                    
            
            elif (case_selector(x3,x2)==2):
                for j in range(16):
                    if(j not in [0,1,12,13,14,15]):
                        new_row15cm[j] = recupx2[j]
                        
                        if (j not in [4,5]): #if the column j is not a pH column
                            new_row5cm[j] = (x3*recupx3[j]+(5-x3)*recupx2[j])/5
                            new_row30cm[j] = ((x2-15)*recupx2[j]+(30-x2)*recupx1[j])/15
                        else:
                            tempx3[j] = concentr(recupx3[j])
                            tempx2[j] = concentr(recupx2[j])
                            tempx1[j] = concentr(recupx1[j])
                            res = (x3*tempx3[j]+(5-x3)*tempx2[j])/5
                            new_row5cm[j] = (-1)*(math.log10(res))
                            res = ((x2-15)*tempx2[j]+(30-x2)*tempx1[j])/15
                            new_row30cm[j] = (-1)*(math.log10(res))
                        
            
            elif (case_selector(x3,x2)==3):
                print('this case is impossible indice 21 de la matrice de verite')
            
            elif (case_selector(x3,x2)==4):
                for j in range(16):
                    if(j not in [0,1,12,13,14,15]):
                        new_row5cm[j] = recupx3[j]
                        new_row30cm[j] = recupx1[j]
                        
                        if (j not in [4,5]): #if the column j is not a pH column
                            new_row15cm[j] = ((x3-5)*recupx3[j]+(x2-x3)*recupx2[j]+(15-x2)*recupx1[j])/10
                        else:
                            tempx3[j] = concentr(recupx3[j])
                            tempx2[j] = concentr(recupx2[j])
                            tempx1[j] = concentr(recupx1[j])
                            res = ((x3-5)*tempx3[j]+(x2-x3)*tempx2[j]+(15-x2)*tempx1[j])/10
                            new_row15cm[j] = (-1)*(math.log10(res))
                            
                        
            elif (case_selector(x3,x2)==5):
                for j in range(16):
                    if(j not in [0,1,12,13,14,15]):
                        new_row5cm[j] = recupx3[j]
                        
                        if (j not in [4,5]): #if the column j is not a pH column
                            new_row15cm[j] = ((x3-5)*recupx3[j]+(15-x3)*recupx2[j])/10
                            new_row30cm[j] = ((x2-15)*recupx2[j]+(30-x2)*recupx1[j])/15
                        else:
                            tempx3[j] = concentr(recupx3[j])
                            #print(recupx2[j],tempx2[j])
                            tempx2[j] = concentr(recupx2[j])
                            tempx1[j] = concentr(recupx1[j])
                            res = ((x3-5)*tempx3[j]+(15-x3)*tempx2[j])/10
                            new_row15cm[j] = (-1)*(math.log10(res))
                            res = ((x2-15)*tempx2[j]+(30-x2)*tempx1[j])/15
                            new_row30cm[j] = (-1)*(math.log10(res))
            
            elif (case_selector(x3,x2)==6):
                print('this case is impossible indice 31 de la matrice de verite')
            elif (case_selector(x3,x2)==7):
                print('this case is impossible indice 32 de la matrice de verite')
                
            elif (case_selector(x3,x2)==8):
                for j in range(16):
                    if(j not in [0,1,12,13,14,15]):
                        new_row5cm[j] = recupx3[j]
                        new_row15cm[j] = recupx3[j]
                        
                        if (j not in [4,5]): #if the column j is not a pH column
                            new_row30cm[j] = ((x3-15)*recupx3[j]+(x2-x3)*recupx2[j]+(30-x2)*recupx1[j])/15
                        else:
                            tempx3[j] = concentr(recupx3[j])
                            tempx2[j] = concentr(recupx2[j])
                            tempx1[j] = concentr(recupx1[j])
                            res = ((x3-15)*tempx3[j]+(x2-x3)*tempx2[j]+(30-x2)*tempx1[j])/15
                            new_row30cm[j] = (-1)*(math.log10(res))
            else:
                print('ce cas nest pas pris en compte')
                print(x1,x2,x3)
                print(profil['profil_id'])
                
                pass
                
            
            new_row30cm['lower_d'] = '30cm'
            new_row30cm_df = new_row30cm.to_frame().transpose()
            profil = pd.concat([profil, new_row30cm_df], ignore_index=True)
            
            
            new_row15cm['lower_d'] = '15cm'
            new_row15cm_df = new_row15cm.to_frame().transpose()
            profil = pd.concat([profil, new_row15cm_df], ignore_index=True)
            
#           updating and replacing each of the 3 additional rows 
            new_row5cm['lower_d'] = '5cm'
            new_row5cm_df = new_row5cm.to_frame().transpose()
            profil = pd.concat([profil, new_row5cm_df], ignore_index=True)
            
            if (x1>=30):
                df_final = pd.concat([df_final, profil.tail(3)], ignore_index=True)
            elif (x1>=15):
                profil.iloc[4,16] = 'valid from 15 cm to ' + str(x1) + ' cm depth and not beyond !' 
                df_final = pd.concat([df_final, profil.tail(2)], ignore_index=True)
            elif (x1>=5):
                profil.iloc[5,16] = 'valid from 5 cm to ' + str(x1) + ' cm depth and not beyond !' 
                df_final = pd.concat([df_final, profil.tail(1)], ignore_index=True)        
            else :
                print('impossible because x3 < x2 < x1 and previous conditions were made on x3 and x2')
                pass
        
        elif (profil.iloc[0,15]==4):
            #print('Nous sommes dans le cas 4')
            pass
            
        else:
            print('ce cas nexiste pas dans notre config', profil.iloc[0,15])
            
#     for profil in list_of_profil:
#         df_final = pd.concat([df_final, profil.tail(3)], ignore_index=True)
#         print(profil.tail(3))
    
    df_final = df_final.drop(['upper_d','horizon_depth','NoH'], axis=1)
    df_final.rename(columns={'lower_d':'ISRIC_depth'}, inplace=True)
    
    sub_df_final = [] #list of the different dataframe per soil parameter
    soil_param = ['clay','bulk_density','ph_h2o','ph_kcl','ca','mg','k','org_carbon','total_p','p_bray1'] #liste des differents dataframes de chaque parametre de sol
    
    for parameter in soil_param:
        transit = df_final[['sid','profil_id',parameter,'ISRIC_depth','Comments']] #extract only the parameter info
        sub_df_final.append(transit)
        transit.to_excel(parameter+'/output.xlsx') #create the corresponding excel file in each folder
        df5cm = transit[transit.ISRIC_depth == '5cm']
        df5cm.to_excel(parameter+'/output5.xlsx')
        df15cm = transit[transit.ISRIC_depth == '15cm']
        df15cm.to_excel(parameter+'/output15.xlsx')
        df30cm = transit[transit.ISRIC_depth == '30cm']
        df30cm.to_excel(parameter+'/output30.xlsx')

    
    
    # this dataframe retrieves all the columns with no NaN occurences, in our case it's an empty dataframe
    null_cols = df_final.columns[df_final.isnull().all()]
    t = df_final.drop(null_cols, axis = 1, inplace = True)
    
    df_final.to_excel('output_harmonized.xlsx')
    
    print('done')

done
